In [1]:
# Si no tenemos el módulo instalado
!pip install langchain-openai
!pip install langchain
!pip install tiktoken
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Memoria - ConversationBufferMemory


In [2]:
from google.colab import userdata


secret_string = userdata.get('OPENAI_TOKEN')
# Configurar el motor de OpenAI
engine = "gpt-4"


In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [4]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=secret_string)
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si ponemos verbose=True devuelve datos de la plantilla
)

In [5]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?'

In [6]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [7]:
conversation.predict(input="¿Cuál es mi nombre?")

'Tu nombre es Carlos.'

In [8]:
print(memory.buffer)

Human: Hola, mi nombre es Carlos
AI: ¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?
Human: ¿Cuál es la capital de Honduras?
AI: La capital de Honduras es Tegucigalpa.
Human: ¿Cuál es mi nombre?
AI: Tu nombre es Carlos.


## ConversationBufferWindowMemory

In [10]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
llm = ChatOpenAI(temperature=0.0, model=engine, openai_api_key=secret_string)
## USamos conversationBufferWindowMemory para almacenar el contexto de la conversación

memory = ConversationBufferWindowMemory(k=4) # Numero de interacciones en AI y usuario

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False # Si usas True podrás ver el prompt real que se está pasando al modelo

)

In [16]:
conversation.predict(input="Hola, mi nombre es Carlos")

'¡Hola, Carlos! Es un placer conocerte. ¿Cómo puedo ayudarte hoy?'

In [17]:
conversation.predict(input="¿Cuál es la capital de Honduras?")

'La capital de Honduras es Tegucigalpa.'

In [18]:
conversation.predict(input="¿Cuál es mi nombre") # No lo recordará ya que k=1. Cambia k=2 para ver que funciona

'Tu nombre es Carlos.'

In [14]:
## ConversationSummaryMemory

In [20]:
from langchain.memory import ConversationSummaryBufferMemory

In [21]:
# creamos un texto
schedule = "Hay una reunión a las 8 a. m. con tu equipo de producto. \
Necesitarás tener preparada tu presentación de PowerPoint. \
De 9 a. m. a 12 p. m. tienes tiempo para trabajar en tu proyecto LangChain, \
que avanzará rápidamente porque LangChain es una herramienta muy potente.\
Al mediodía, almuerzo en el restaurante italiano con un cliente que viene conduciendo\
desde más de una hora de distancia para reunirse contigo y entender lo último en IA.\
Asegúrate de traer tu portátil para mostrar la última demo de LLM."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hola"}, {"output": "¿Qué hay?"})
memory.save_context({"input": "Hoy es el último día para reservar plaza en el congreso"},
                    {"output": "Guay"})
memory.save_context({"input": "Qué hay en la agenda hoy?"},
                    {"output": f"{schedule}"})

In [22]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI and informs it that today is the last day to reserve a spot at the congress. The AI responds casually. The human then asks about the day's agenda. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting at an Italian restaurant with a client who is interested in the latest in AI. The AI advises the human to bring a laptop to show the latest LLM demo."}

In [23]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [25]:
#conversation.predict(input="Qué hay en la agenda para hoy?")
#conversation.predict(input="Algún evento importante que deba recordar?")
conversation.predict(input="Que plan hay para hoy?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and informs it that today is the last day to reserve a spot at the congress. The AI responds casually. The human then asks about the day's agenda. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting at an Italian restaurant with a client who is interested in the latest in AI. The AI advises the human to bring a laptop to show the latest LLM demo. The human asks for interesting topics to discuss during the lunch meeting. The AI suggests discussing the latest trends in AI, such as deep learning and natural language processing, how their latest LLM demo can benefit 

'Hoy tenemos una agenda bastante ocupada. Comenzaremos con una reunión matutina con el equipo de producto. Luego, tendrás tiempo para trabajar en el proyecto LangChain. Al mediodía, tenemos una reunión de almuerzo en un restaurante italiano con un cliente que está interesado en lo último en IA. Te recomendaría que lleves tu laptop para mostrar la última demostración de LLM. Durante la reunión, podrías discutir sobre las últimas tendencias en IA, como el aprendizaje profundo y el procesamiento del lenguaje natural, cómo nuestra última demostración de LLM puede beneficiar el negocio del cliente, casos de uso exitosos de nuestra tecnología en negocios similares, y los desafíos actuales del cliente y cómo nuestras soluciones podrían ayudar a abordarlos.'